In [ ]:
# Medical Dispensing Machine Program Start
print("                             * Medical Dispensing Machine Program *")

In [ ]:
# Import necessary libraries
import mysql.connector # Establishes a connection to the MySQL database
import pandas as pd # Handles and analyses patient data in table format
from datetime import datetime # Manages date and time operations (age calculation)

In [ ]:
# Connect to MySQL Database
try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="medicaldispensingdb"
    )
   
    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()

    # Display message confirming successful connection
    print("Connected to `medicaldispensingdb` successfully.")
    
except mysql.connector.Error as e:
    print(f"Database connection error: {e}")


In [ ]:
# Retrieve Patient Information from MySQL Database
# This SQL query retrieves patient details, including medical and carer information.

query = '''
SELECT 
    Patients.PatientID, 
    Patients.Name, 
    Patients.DateOfBirth, 
    Patients.Gender, 
    Patients.Age,
    Medication.HealthCondition,
    MedicationTypes.MedName AS MedicationName, 
    Medication.DosageTime, 
    Medication.Status,
    Medication.Colour,
    Medication.MedicationStartDate,
    Medication.MedicationEndDate,
    Medication.CourseType,
    Carers.PhoneNumber, 
    Carers.Email, 
    Carers.TypeOfCarer, 
    Carers.PreferredContactMethod
FROM Patients
JOIN Medication ON Patients.PatientID = Medication.PatientID
JOIN MedicationTypes ON Medication.MedTypeID = MedicationTypes.MedTypeID
JOIN Carers ON Patients.PatientID = Carers.PatientID;

'''

# Execute the SQL Query to retrieve patient records
cursor.execute(query)
results = cursor.fetchall()

# Load into DataFrame with correct columns
data = pd.DataFrame(results, columns=[
    'Patient ID', 'Name', 'Date of Birth', 'Gender', 'Age', 'Health Condition', 'Medication Name',
    'Dosage Time', 'Status', 'Colour', 'Medication Start Date', 'Medication End Date', 'Course Type',
    'Phone Number', 'Email', 'Type of Carer', 'Preferred Contact Method'
])

# Display message confirming successful data retrieval
print("* Patient Data Loaded Successfully *\n")

In [ ]:
# Dataset Summary
num_rows, num_columns = data.shape
print("\n\n * Dataset Overview *\n")
print(f"The dataset contains {num_rows} records and {num_columns} columns.\n")


In [ ]:
# Convert 'Date of Birth' to datetime and calculate age
data['Date of Birth'] = pd.to_datetime(data['Date of Birth'], format='%Y-%m-%d', errors='coerce')
current_year = datetime.now().year
data['Age'] = data['Date of Birth'].apply(lambda dob: current_year - dob.year if not pd.isnull(dob) else None)

# Calculate the patient's age based on the current year
current_year = datetime.now().year
data['Age'] = data['Date of Birth'].apply(lambda dob: current_year - dob.year if not pd.isnull(dob) else None)

# Display confirmation message
print("Age calculation completed successfully.\n")

# Define a function to display patient information
# This function takes a Patient ID as input, searches for the patient in the dataset,and displays their details if found.

def display_patient_info(patient_id):
    print(f"Retrieving information for Patient ID: {patient_id}\n")
    # Search for the patient's information in the dataset
    patient_info = data[data['Patient ID'] == int(patient_id)]
    # If no matching patient is found, ask the user to enter a valid ID
    if patient_info.empty:
        print(f"No patient found with ID. Please enter an exisiting Patient ID: {patient_id}\n")
        return
        
    # Loop through the retrieved patient records and display the information
    for _, row in patient_info.iterrows():
        print("********************************")
        print(f"Patient ID: {row['Patient ID']}")
        print(f"Patient Name: {row['Name']}")
        print(f"Age: {row['Age']}")
        print(f"Gender: {row['Gender']}")
        print(f"Medication: {row['Medication Name']}")
        print(f"Dosage Time: {row['Dosage Time']}")
        print(f"Status: {row['Status']}")
        print(f"Email: {row['Email']}")
        print(f"Phone Number: {row['Phone Number']}")
        print(f"Type of Carer: {row['Type of Carer']}")
        print(f"Preferred Contact Method: {row['Preferred Contact Method']}")


In [ ]:
# Patient Information Retrieval
# This function retrieves patient details, medication history, and carer information from the database.

print("* Patient Information Retrieval *")

def display_patient_info(patient_id):
    # Ensure the database connection is active before querying
    if conn is None:
        print("Database not connected. Patient data cannot be retrieved.")
        return

    try:
        # Make sure the patient exists in the database before running the query
        cursor.execute("SELECT COUNT(*) FROM Patients WHERE PatientID = %s", (patient_id,))
        if cursor.fetchone()[0] == 0:
            print(f"No patient found with ID: {patient_id}")
            return

        # Retrieve patient details, medication and carer information
        query = '''
        SELECT Patients.PatientID, Patients.Name, Patients.DateOfBirth, Patients.Gender, 
               Patients.Age, Medication.HealthCondition, MedicationTypes.MedName AS MedicationName, 
               Medication.DosageTime, Medication.Status, Medication.Colour, 
               Medication.MedicationStartDate, Medication.MedicationEndDate, Medication.CourseType, 
               Carers.PhoneNumber, Carers.Email, Carers.TypeOfCarer, Carers.PreferredContactMethod
        FROM Patients
        JOIN Medication ON Patients.PatientID = Medication.PatientID
        JOIN MedicationTypes ON Medication.MedTypeID = MedicationTypes.MedTypeID
        JOIN Carers ON Patients.PatientID = Carers.PatientID
        WHERE Patients.PatientID = %s
        '''
        
        # Execute the query and fetch patient information
        cursor.execute(query, (patient_id,))
        patient_info = cursor.fetchall()
        
        # If patient details are found, display them in a structured format
        for row in patient_info:
            print("********************************")
            print(f"Patient ID: {row[0]}")
            print(f"Patient Name: {row[1]}")
            print(f"Date of Birth: {row[2]}")
            print(f"Gender: {row[3]}")
            print(f"Age: {row[4]}")
            print(f"Health Condition: {row[5]}")
            print(f"Medication: {row[6]}")
            print(f"Dosage Time: {row[7]}")
            print(f"Status: {row[8]}")
            print(f"Colour: {row[9]}")
            print(f"Medication Start Date: {row[10]}")
            print(f"Medication End Date: {row[11]}")
            print(f"Course Type: {row[12]}")
            print(f"Phone Number: {row[13]}")
            print(f"Email: {row[14]}")
            print(f"Type of Carer: {row[15]}")
            print(f"Preferred Contact Method: {row[16]}")
            print("********************************\n")

    except mysql.connector.Error as e:
        print(f"MySQL Error: {e}")  # Handles errors related to database connection or query issues
    except Exception as e:
        print(f"An unexpected error occurred: {e}") # Handles any other unexpected issues

# Input Loop for Patient Retrieval
# This loop continuously asks the user for a Patient ID and retrieves details until 'exit' is entered.
while True:
    patient_id = input(" Please enter a Patient ID to retrieve details (or 'exit' to quit): ").strip()
    if patient_id.lower() == 'exit':
        print("Patient Retrieval Complete")
        break
    display_patient_info(patient_id)
